In [ ]:
import logging
import gzip
import json
import os
import pickle
import numpy as np

logging.basicConfig(level=logging.WARNING)

try:
    import keras
    from keras.datasets import mnist
    from keras.models import Sequential
    from keras.layers import Dense, Dropout, Flatten
    from keras.layers import Conv2D, MaxPooling2D
    from keras import backend as K
except:
    raise ImportError("For this example you need to install keras.")

import hpbandster.core.nameserver as hpns

from hpbandster.optimizers import RandomSearch

import ConfigSpace as CS
import ConfigSpace.hyperparameters as CSH
from hpbandster.core.worker import Worker
import argparse

#from cnn_mnist_solution import mnist

def onehot(labels):
    """this creates a one hot encoding from a flat vector:
    i.e. given y = [0,2,1]
     it creates y_one_hot = [[1,0,0], [0,0,1], [0,1,0]]
    """
    classes = np.unique(labels)
    n_classes = classes.size
    one_hot_labels = np.zeros(labels.shape + (n_classes,))
    for c in classes:
        one_hot_labels[labels == c, c] = 1
    return one_hot_labels

def mnist(datasets_dir='./data'):
    if not os.path.exists(datasets_dir):
        os.mkdir(datasets_dir)
    data_file = os.path.join(datasets_dir, 'mnist.pkl.gz')
    if not os.path.exists(data_file):
        print('... downloading MNIST from the web')
        try:
            import urllib
            urllib.urlretrieve('http://google.com')
        except AttributeError:
            import urllib.request as urllib
        url = 'http://www.iro.umontreal.ca/~lisa/deep/data/mnist/mnist.pkl.gz'
        urllib.urlretrieve(url, data_file)

    print('... loading data')
    # Load the dataset
    f = gzip.open(data_file, 'rb')
    try:
        train_set, valid_set, test_set = pickle.load(f, encoding="latin1")
    except TypeError:
        train_set, valid_set, test_set = pickle.load(f)
    f.close()

    test_x, test_y = test_set
    test_x = test_x.astype('float32')
    test_x = test_x.astype('float32').reshape(test_x.shape[0], 28, 28, 1)
    test_y = test_y.astype('int32')
    valid_x, valid_y = valid_set
    valid_x = valid_x.astype('float32')
    valid_x = valid_x.astype('float32').reshape(valid_x.shape[0], 28, 28, 1)
    valid_y = valid_y.astype('int32')
    train_x, train_y = train_set
    train_x = train_x.astype('float32').reshape(train_x.shape[0], 28, 28, 1)
    train_y = train_y.astype('int32')
    print('... done loading data')
    return train_x,onehot(train_y), valid_x, onehot(valid_y), test_x,onehot(test_y)

def train_again(config,budget):
    model = Sequential()

    model.add(Conv2D(config['num_filters'], kernel_size=(config['filter_size'],config['filter_size']),
                         activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(config['num_filters'], kernel_size=(config['filter_size'], config['filter_size']),
                         activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
      
    model.add(Dropout(0.4))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.4))
    model.add(Dense(10, activation='softmax'))

    optimizer = keras.optimizers.SGD(lr=config['learning_rate'])

    model.compile(loss=keras.losses.categorical_crossentropy,optimizer=optimizer,metrics=['accuracy'])

    history=model.fit(self.x_train, self.y_train,
                  batch_size=config["batch_size"],epochs=epochs,verbose=0,validation_data=(self.x_test, self.y_test))

    return history
    
class MyWorker(Worker):

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.x_train, self.y_train, self.x_valid, self.y_valid, self.x_test, self.y_test = mnist("./")

    def compute(self, config, budget,**kwargs):
        """
        Evaluates the configuration on the defined budget and returns the validation performance.
        Args:
            config: dictionary containing the sampled configurations by the optimizer
            budget: (float) amount of time/epochs/etc. the model can use to train
        Returns:
            dictionary with mandatory fields:
                'loss' (scalar)
                'info' (dict)
        """
        lr = config["learning_rate"]
        num_filters = config["num_filters"]
        batch_size = config["batch_size"]
        epochs = budget

        # TODO: train and validate your convolutional neural networks here
        
        model = Sequential()

        model.add(Conv2D(config['num_filters'], kernel_size=(config['filter_size'],config['filter_size']),
                         activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))

        model.add(Conv2D(config['num_filters'], kernel_size=(config['filter_size'], config['filter_size']),
                         activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        
        
        
        model.add(Dropout(0.4))
        model.add(Flatten())
        model.add(Dense(128, activation='relu'))
        model.add(Dropout(0.4))
        model.add(Dense(10, activation='softmax'))


        optimizer = keras.optimizers.SGD(lr=config['learning_rate'])

        model.compile(loss=keras.losses.categorical_crossentropy,optimizer=optimizer,metrics=['accuracy'])

        history=model.fit(self.x_train, self.y_train,
                  batch_size=config["batch_size"],epochs=epochs,verbose=0,validation_data=(self.x_test, self.y_test))

        train_score = model.evaluate(self.x_train, self.y_train, verbose=0)
        val_score = model.evaluate(self.x_valid, self.y_valid, verbose=0)
        test_score = model.evaluate(self.x_test, self.y_test, verbose=0)
        validation_error=1-val_score[1]

        # TODO: We minimize so make sure you return the validation error here
        return ({
            'loss': validation_error,  # this is the a mandatory field to run hyperband
            'info': {'test accuracy': test_score[1],
                        'train accuracy': train_score[1],
                        'validation accuracy': val_score[1],
                        'number of parameters': model.count_params(),
                
            }  # can be used for any user-defined information - also mandatory
        })

    @staticmethod
    def get_configspace():
        
        # TODO: Implement configuration space here. 
        #See https://github.com/automl/HpBandSter/blob/master/hpbandster/examples/example_5_keras_worker.py  for an example
      
        cs = CS.ConfigurationSpace()

        lr = CSH.UniformFloatHyperparameter('learning_rate', lower=0.0001, upper=0.1, default_value='0.1', log=True)

        # For demonstration purposes, we add different optimizers as categorical hyperparameters.
        # To show how to use conditional hyperparameters with ConfigSpace, we'll add the optimizers 'Adam' and 'SGD'.
        # SGD has a different parameter 'momentum'.
        #optimizer = CSH.CategoricalHyperparameter('optimizer', ['Adam', 'SGD'])

        #sgd_momentum = CSH.UniformFloatHyperparameter('sgd_momentum', lower=0.0, upper=0.99, default_value=0.9, log=False)

        #cs.add_hyperparameters([lr, optimizer, sgd_momentum])
        cs.add_hyperparameters([lr])
        
        batch_size = CSH.UniformIntegerHyperparameter('batch_size', lower=16, upper=128, default_value=16, log=True)
        cs.add_hyperparameters([batch_size])

        #num_conv_layers =  CSH.UniformIntegerHyperparameter('num_conv_layers', lower=1, upper=3, default_value=2)

        num_filters = CSH.UniformIntegerHyperparameter('num_filters', lower=8, upper=64, default_value=16, log=True)
        cs.add_hyperparameters([num_filters])
        
        filter_size = CSH.CategoricalHyperparameter('filter_size', [3,5])
        cs.add_hyperparameters([filter_size])


        #dropout_rate = CSH.UniformFloatHyperparameter('dropout_rate', lower=0.0, upper=0.9, default_value=0.5, log=False)
        #num_fc_units = CSH.UniformIntegerHyperparameter('num_fc_units', lower=8, upper=256, default_value=32, log=True)

        #cs.add_hyperparameters([dropout_rate, num_fc_units])


        # The hyperparameter sgd_momentum will be used,if the configuration
        # contains 'SGD' as optimizer.
        #cond = CS.EqualsCondition(sgd_momentum, optimizer, 'SGD')
        #cs.add_condition(cond)

        # You can also use inequality conditions:
        #cond = CS.GreaterThanCondition(num_filters_2, num_conv_layers, 1)
        #cs.add_condition(cond)

        #cond = CS.GreaterThanCondition(num_filters_3, num_conv_layers, 2)
        #cs.add_condition(cond)

        return cs

       




parser = argparse.ArgumentParser(description='Example 1 - sequential and local execution.')
parser.add_argument('--budget', type=float,
                    help='Maximum budget used during the optimization, i.e the number of epochs.', default=12)
parser.add_argument('--n_iterations', type=int, help='Number of iterations performed by the optimizer', default=20)
#args = parser.parse_args()

# Step 1: Start a nameserver
# Every run needs a nameserver. It could be a 'static' server with a
# permanent address, but here it will be started for the local machine with the default port.
# The nameserver manages the concurrent running workers across all possible threads or clusternodes.
# Note the run_id argument. This uniquely identifies a run of any HpBandSter optimizer.
NS = hpns.NameServer(run_id='example1', host='127.0.0.1', port=None)
NS.start()

# Step 2: Start a worker
# Now we can instantiate a worker, providing the mandatory information
# Besides the sleep_interval, we need to define the nameserver information and
# the same run_id as above. After that, we can start the worker in the background,
# where it will wait for incoming configurations to evaluate.
w = MyWorker(nameserver='127.0.0.1', run_id='example1')
w.run(background=True)

# Step 3: Run an optimizer
# Now we can create an optimizer object and start the run.
# Here, we run RandomSearch, but that is not essential.
# The run method will return the `Result` that contains all runs performed.

rs = RandomSearch(configspace=w.get_configspace(),
                  run_id='example1', nameserver='127.0.0.1',
                 # min_budget=int(args.budget), max_budget=int(args.budget))
                  min_budget=int(6), max_budget=int(6))
res = rs.run(n_iterations=50)
# Step 4: Shutdown
# After the optimizer run, we must shutdown the master and the nameserver.
rs.shutdown(shutdown_workers=True)
NS.shutdown()

# Step 5: Analysis
# Each optimizer returns a hpbandster.core.result.Result object.
# It holds information about the optimization run like the incumbent (=best) configuration.
# For further details about the Result object, see its documentation.
# Here we simply print out the best config and some statistics about the performed runs.
id2config = res.get_id2config_mapping()
incumbent = res.get_incumbent_id()

print('Best found configuration:', id2config[incumbent]['config'])


# Plots the performance of the best found validation error over time
all_runs = res.get_all_runs()
# Let's plot the observed losses grouped by budget,
import hpbandster.visualization as hpvis

hpvis.losses_over_time(all_runs)


import matplotlib.pyplot as plt
plt.savefig("random_search.png")

# TODO: retrain the best configuration (called incumbent) and compute the test error
#worker = MyWorker(nameserver='127.0.0.1', run_id='example1')
#res = worker.compute(config=id2config[incumbent]['config'], budget=6, working_directory='.',test=True)
res=train_again(config=id2config[incumbent]['config'], budget=6)
print("Test Results:")
val_loss = res.history['val_loss']
plt.plot(loss)
print(res.history['info'])
#© 

C:\Users\MYPC\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Users\MYPC\Anaconda3\lib\site-packages\Pyro4\util.py:839: UserWarning: msgpack serializer unavailable. requires msgpack 0.5.2+, found (0, 5, 1)
  warnings.warn("msgpack serializer unavailable. requires msgpack 0.5.2+, found " + str(msgpack.version))
C:\Users\MYPC\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


... loading data
... done loading data
